In [12]:
import json
import boto3
import sagemaker
from sagemaker.tensorflow import TensorFlow
print("boto3 : ", boto3.__version__)
print("sagemaker : ", sagemaker.__version__)

boto3 :  1.14.44
sagemaker :  2.16.1


In [13]:
# utilities
get_s3_path = lambda *args: "s3://" + "/".join(args)

In [14]:
# 非公開にしたい情報を読み込む(今回はRoleのARNのみ)
with open("./secrets.json", "r", encoding="utf-8") as fp:
    secrets = json.load(fp)
role=secrets["RoleArn"]
s3_bucket=secrets["S3Bucket"]
print("S3 bucket for training : ", s3_bucket)

S3 bucket for training :  sagemaker-nst


In [15]:
# AWSとのセッションを確立する
boto_session = boto3.Session(region_name="ap-northeast-1")
sess = sagemaker.Session(
    # リージョンを東京リージョンに指定
    boto_session=boto_session,
    # SageMkaerが使用するS3バケットを指定
    default_bucket=s3_bucket,
)

In [19]:
# トレーニング設定の定義に必要なパラメータを用意
# トレーニングで使用されるデータのダウンロード元、トレーニングで生成されるデータのアップロード先となるS3のパスを指定
s3_train_path = get_s3_path(sess.default_bucket(), "train")

# インスタンスタイプに"local"を指定すると、トレーニングジョブの実行環境のDockerイメージがpullされ、
# ローカル上でコンテナが起動しトレーニングが実行される
instance_types = {"CPU" : "ml.m5.large", "GPU" : "ml.g4dn.xlarge", "LOCAL" : "local"}

# トレーニングスクリプトに渡されるパラメータ
hyperparameters = {
    "EPOCH" : 50,
    "STEP" : 10,
    "MAX_IMAGE_SIZE" : 1024,
    "TB_BUCKET" : s3_train_path,
    "MAX_TRIAL" : 36,
}

In [17]:
# TensorFlowで書かれたモデルのトレーニングジョブ設定を定義
est = TensorFlow(
    # トレーニングスクリプト
    entry_point='train.py',
    # 指定したS3にSageMakerがアクセスするためのサービスロール
    role=role,
    # 起動されるインスタンス数
    instance_count=1,
    # 起動されるインスタンスのタイプ
    instance_type=instance_types["GPU"],
    # トレーニングジョブ実行環境内で使用されるtensorflowのバージョン
    framework_version='2.3.0',
    # トレーニングジョブ実行環境内で使用されるPythonのバージョン
    py_version='py37',
    # デバッグライブラリの使用を無効化。理由は後述
    debugger_hook_config=False,
    # JSON形式で指定しtパラメータは、トレーニングジョブ実行時にコマンドライン引数としてトレーニングスクリプトに渡される。
    hyperparameters=hyperparameters,
    # ユーザが指定したS3バケットやリージョンを使用したいので、カスタマイズしたセッション情報も渡す
    sagemaker_session=sess,
    # スポットインスタンスの使用を有効にする。nstモデルの場合、料金を約70%節約できる。
    use_spot_instances=True,
    max_run=7200,
    max_wait=7200,
)

In [18]:
# トレーニングジョブを実行
# 指定したS3バケットのパスから訓練用データがトレーニングジョブ実行環境上に自動ダウンロードされる。
# デフォルト(logs="All")だとログが大量に出すぎるのでここでは抑制する。
est.fit(s3_train_path, logs="None", wait=False,)

In [10]:
# # tensorboard でトレーニング結果を確認
# %load_ext tensorboard
# %tensorboard --logdir s3://sagemaker-nst/train --port 6006 --host 0.0.0.0